# Upload project

How to upload data to Supervisely using API

**Input**:
- Local folder with Project in Supervisely format

**Output**:
- New Project in Supervisely

## Configuration

Edit the following settings for your own case

In [1]:
import supervisely_lib as sly
import os
import json
from tqdm import tqdm

In [2]:
# Local folder with Project
project_directory = './tutorial_project'

# New Project
team_name = "jupyter_tutorials"
workspace_name = "cookbook"
project_name = "uploaded_project"

# Obtain server address and your api_token from environment variables
# Edit those values if you run this notebook on your own PC
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [3]:
# Initialize API object
api = sly.Api(address, token)

## Verify input values

Test that context (team / workspace / project) exists

In [4]:
# get IDs of team, workspace and project by names

team = api.team.get_info_by_name(team_name)
if team is None:
    raise RuntimeError("Team {!r} not found".format(team_name))

workspace = api.workspace.get_info_by_name(team.id, workspace_name)
if workspace is None:
    raise RuntimeError("Workspace {!r} not found".format(workspace_name))
    
print("Team: id={}, name={}".format(team.id, team.name))
print("Workspace: id={}, name={}".format(workspace.id, workspace.name))

Team: id=30, name=jupyter_tutorials
Workspace: id=76, name=cookbook


## Read local Project

In [5]:
# read project from directory 
project_fs = sly.Project(project_directory, sly.OpenMode.READ)

## Create remote Project

In [6]:
# check if project already exists. If yes - generate new free name
if api.project.exists(workspace.id, project_name):
    project_name = api.project.get_free_name(workspace.id, project_name)

In [7]:
# create remote project and set corresponding meta information
project = api.project.create(workspace.id, project_name)
api.project.update_meta(project.id, project_fs.meta.to_json())
print("Project: id={}, name={}".format(project.id, project.name))

Project: id=1337, name=uploaded_project


In [8]:
# iterate over datasets, images and thier annotations in directory and upload that data to remote server
for dataset_fs in project_fs:
    dataset = api.dataset.create(project.id, dataset_fs.name)
    
    names, img_paths, ann_paths = [], [], []
    for item_name in dataset_fs:
        img_path, ann_path = dataset_fs.get_item_paths(item_name)
        names.append(item_name)
        img_paths.append(img_path)
        ann_paths.append(ann_path)
    
    print("Dataset: {}. Will upload {} images with annotations".format(dataset.name, len(img_paths)), flush=True)
    with tqdm(total=len(names), desc="Upload images") as progress_bar:
        img_infos = api.image.upload_paths(dataset.id, names, img_paths, progress_bar.update)
    
    image_ids = [img_info.id for img_info in img_infos]
    with tqdm(total=len(names), desc="Upload annotations") as progress_bar:
        api.annotation.upload_paths(image_ids, ann_paths, progress_bar.update)     

Dataset: dataset_01. Will upload 3 images with annotations


Upload annotations: 100%|██████████| 3/3 [00:00<00:00, 22.74it/s]


Dataset: dataset_02. Will upload 2 images with annotations


Upload annotations: 100%|██████████| 2/2 [00:00<00:00, 29.52it/s]


In [9]:
print("Project {!r} has been sucessfully uploaded".format(project_name))
print("Number of uploaded images: ", api.project.get_images_count(project.id))

Project 'uploaded_project' has been sucessfully uploaded
Number of uploaded images:  5
